In [1]:
import yaml
import pandas as pd
 
try:
    with open("../config.yaml", "r") as file:
        config = yaml.safe_load(file)

    df = pd.read_csv(config['data']['clean_data']['full_clean'], sep=";")

except:
    print("Yaml configuration file not found!")

In [2]:
df.head()

,airline,flight,source_city,departure_time,stops,arrival_time,destination_city,class,duration,lead_time_days,price
0,SpiceJet,SG-8709,Delhi,Evening,zero,Night,Mumbai,Economy,2h 17m,1,5953
1,SpiceJet,SG-8157,Delhi,Early_Morning,zero,Morning,Mumbai,Economy,2h 33m,1,5953
2,AirAsia,I5-764,Delhi,Early_Morning,zero,Early_Morning,Mumbai,Economy,2h 17m,1,5956
3,Vistara,UK-995,Delhi,Morning,zero,Afternoon,Mumbai,Economy,2h 25m,1,5955
4,Vistara,UK-963,Delhi,Morning,zero,Morning,Mumbai,Economy,2h 33m,1,5955


In [3]:
# Get unique airline
airline_name_df = pd.DataFrame({"airline_name_id": range(1, df["airline"].nunique() + 1),
                          "airline_name": df["airline"].drop_duplicates().reset_index(drop=True)})

# Get unique flight_number
flight_number_df = pd.DataFrame({"flight_number_id": range(1, df["flight"].nunique() + 1),
                               "flight_number": df["flight"].drop_duplicates().reset_index(drop=True)})

In [4]:
airline_name_df

,airline_name_id,airline_name
0,1,SpiceJet
1,2,AirAsia
2,3,Vistara
3,4,GO_FIRST
4,5,Indigo
5,6,Air_India


In [5]:
flight_number_df

,flight_number_id,flight_number
0,1,SG-8709
1,2,SG-8157
2,3,I5-764
3,4,UK-995
4,5,UK-963
...,...,...
1556,1557,6E-7141
1557,1558,6E-557
1558,1559,6E-7127
1559,1560,6E-7259


In [6]:
# Get unique class
class_df = pd.DataFrame({"class_id": range(1, df["class"].nunique() + 1),
                          "class": df["class"].drop_duplicates().reset_index(drop=True)})

# Get unique price
price_df = pd.DataFrame({"price_id": range(1, df["price"].nunique() + 1),
                               "price": df["price"].drop_duplicates().reset_index(drop=True)})

In [7]:
class_df

,class_id,class
0,1,Economy
1,2,Business


In [8]:
price_df

,price_id,price
0,1,5953
1,2,5956
2,3,5955
3,4,6060
4,5,5954
...,...,...
12152,12153,91531
12153,12154,77217
12154,12155,87051
12155,12156,74731


In [9]:
# Merge airline_name_df
ticket_df = df.merge(airline_name_df, how="left", left_on="airline", right_on="airline_name")

# Merge flight_number_df
ticket_df = ticket_df.merge(flight_number_df, how="left", left_on="flight", right_on="flight_number")

# Merge class_df
ticket_df = ticket_df.merge(class_df, how="left", left_on="class", right_on="class")

# Merge price_df
ticket_df = ticket_df.merge(price_df, how="left", left_on="price", right_on="price")

In [10]:
#Keeping only id columns and dropping duplicates
ticket_df = ticket_df[[
    "airline_name_id", "airline_name",
    "flight_number_id", "flight_number",
    "class_id", "class",
    "price_id", "price"
]].drop_duplicates().reset_index(drop=True)

In [11]:
#Adding primary key (ticket_id)
ticket_df["ticket_id"] = range(1, len(ticket_df) + 1)

In [12]:
#Reordering columns
ticket_df = ticket_df[[
    "ticket_id",
    "airline_name_id", "airline_name",
    "flight_number_id", "flight_number",
    "class_id", "class",
    "price_id", "price"
]]

ticket_df

,ticket_id,airline_name_id,airline_name,flight_number_id,flight_number,class_id,class,price_id,price
0,1,1,SpiceJet,1,SG-8709,1,Economy,1,5953
1,2,1,SpiceJet,2,SG-8157,1,Economy,1,5953
2,3,2,AirAsia,3,I5-764,1,Economy,2,5956
3,4,3,Vistara,4,UK-995,1,Economy,3,5955
4,5,3,Vistara,5,UK-963,1,Economy,3,5955
...,...,...,...,...,...,...,...,...,...
41047,41048,3,Vistara,1462,UK-826,2,Business,12157,77105
41048,41049,3,Vistara,1438,UK-832,2,Business,12146,79099
41049,41050,3,Vistara,1462,UK-826,2,Business,12147,81585
41050,41051,6,Air_India,1456,AI-569,2,Business,12140,68739


In [ ]:
#Saving new table to csv
#ticket_df.to_csv(config['data']['sql']['ticket'], index=False, encoding="utf-8")